### Quiz( data/wine-data.csv )
* 각 분류기들을 이용하여 모델을 생성하고 평가하시오
* class : 0 - 레드와인, 1 - 화이트와인


### Quiz

* 다음 데이터셋(loan_train_cleaning.csv)을 분석하여 loan_test_cleaning.csv파일의 레이블을 넣으시오
> test데이터에 레이블이 없어 처리하려 한다. 또한 데이터 자동화를 하여 대출 가능한지 아닌지를 만들고자 한다
* Dream Housing Finance 회사는 모든 주택 융자를 취급합니다. 그들은 모든 도시, 중 도시 및 농촌 지역에 존재합니다.
* 해당 회사가 고객의 대출 자격을 확인한 후 주택 대출을 고객에게 해준다.
* 고객은 지원 양식을 작성하는 동안 회사는 고객 세부 정보를 기반으로 대출 자격 프로세스(실시간)를  자동화하려고 합니다. 
* 이러한 세부 정보는 성별, 결혼 여부, 교육, 부양 가족 수, 소득, 대출 금액, 신용 기록 등입니다. 
* 특정 고객이 각각의 값을 입력 했을 경우 대출이 되는지 안되는지 자동화를 진행하고자 한다.
>
* 컬럼
    - Loan_ID : 고유 대출 id
    - Gender : 성별(1:남성, 0:여성)
    - married : 1:기혼, 0:미혼
    - dependents : 부양가족 수
    - education : 대학원( 0:졸업, 1:미졸업 )
    - self_employed : 0:자영업 아님, 1:자영업
    - applicantlncome : 신청자 소득
    - coapplicantlncome : 신청인 소득
    - loanAmount : 대출금액(천만원)
    - loan_amount_term : 대출기간(개월)
    - credit_history : 신용 기록 지침(0:미충족, 1:충족)
    - property_area : 2:도시, 1:중 도시, 0:농촌
    - loan_status : 대출(1:승인, 0:거절)

In [1]:
import pandas as pd
wine = pd.read_csv('data/wine-data.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0
1,9.8,2.6,3.20,0
2,9.8,2.3,3.26,0
3,9.8,1.9,3.16,0
4,9.4,1.9,3.51,0


In [2]:
wine.columns

Index(['alcohol', 'sugar', 'pH', 'class'], dtype='object')

In [3]:
features = ['alcohol', 'sugar', 'pH']
label = 'class'

X, y = wine[features], wine[label]


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = \
                train_test_split(X, y ,test_size=0.2)

In [5]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier()
kn.fit(X_train, y_train)

print(kn.score(X_train, y_train))
print(kn.score(X_test, y_test))

0.9045603232634212
0.8623076923076923


In [6]:
from sklearn.svm import SVC
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)
print(svm_linear.score(X_train, y_train))
print(svm_linear.score(X_test, y_test))

0.7796805849528574
0.7623076923076924


In [7]:
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train, y_train)
print(svm_rbf.score(X_train, y_train))
print(svm_rbf.score(X_test, y_test))

0.8449105253030594
0.8430769230769231


In [9]:
svm_rbf = SVC(kernel='rbf', probability=True)
svm_rbf.fit(X_train, y_train)
from sklearn.ensemble import VotingClassifier
vo = VotingClassifier(estimators=[('svc',svm_rbf),('knn',kn)], voting='soft')

vo.fit( X_train, y_train )
print('svm : ', svm_rbf.score(X_test, y_test) )
print('knn : ', kn.score(X_test, y_test) )
print('vo : ', vo.score(X_test, y_test ))


svm :  0.8430769230769231
knn :  0.8623076923076923
vo :  0.8661538461538462


### MinMaxScaler


In [11]:
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler()
minMaxScaler.fit(X_train)
X_train_minMax = minMaxScaler.transform(X_train)
X_test_minMax = minMaxScaler.transform(X_test)

### StandardScaler

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaler = scaler.transform(X_train)
X_test_scaler = scaler.transform(X_test)

In [15]:
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train_minMax, y_train)
print(svm_linear.score(X_train_minMax, y_train))
print(svm_linear.score(X_test_minMax, y_test))

0.7558206657687128
0.7446153846153846


In [16]:
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train_minMax, y_train)
print(svm_rbf.score(X_train_minMax, y_train))
print(svm_rbf.score(X_test_minMax, y_test))

0.800461804887435
0.7807692307692308


In [18]:
#GridSearchCV
from sklearn.model_selection import GridSearchCV

param_c = [ 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
param_gamma = [ 0.001, 0.01, 0.1, 1.0, 10.0 ]
params = {
    'C':param_c,
    'gamma':param_gamma
}

svc = SVC(kernel = 'rbf')

grid_cv = GridSearchCV(svc, param_grid = params, cv=3, n_jobs = -1)# n_jobs : 사용할 코어의 수 . -1인경우 모든 코어를 사용함
grid_cv.fit(X_train_scaler, y_train)
print('최적의 하이퍼 파라미터 : ', grid_cv.best_params_)

print('train :',grid_cv.score(X_train_scaler,y_train))
print('test :',grid_cv.score(X_test_scaler,y_test))

최적의 하이퍼 파라미터 :  {'C': 100.0, 'gamma': 1.0}
train : 0.8807004040792765
test : 0.85


In [19]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

rfc.fit(X_train, y_train)

print("train : ",rfc.score(X_train,y_train))
print("test : ",rfc.score(X_test,y_test))

train :  0.996921300750433
test :  0.8976923076923077


### GradientBoosting

In [20]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)

print("train : ",gbc.score(X_train,y_train))
print("test : ",gbc.score(X_test,y_test))

train :  0.8862805464691168
test :  0.8569230769230769
